In [ ]:
# stdlib
import os
from pathlib import Path

# syft absolute
import syft as sy
from syft.service.code.user_code import UserCode
from syft.service.user.user import User

print("upgrading to", sy.__version__)

In [ ]:
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# os.environ["SERVER_PORT"] = "8081"

In [ ]:
environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
server_port = os.environ.get("SERVER_PORT", "auto")
print(environment, server_port)

# Login

In [ ]:
server = sy.orchestra.launch(
    name="test_upgradability",
    dev_mode=True,
    reset=True,
    port=server_port,
    n_consumers=1,
    create_producer=True,
)

client = server.login(email="info@openmined.org", password="changethis")

In [ ]:
# Check if this is a new server
migration_data = client.get_migration_data()

assert len(migration_data.store_objects[User]) == 1
assert UserCode not in migration_data.store_objects
assert not migration_data.includes_custom_workerpools

# Load migration data

In [ ]:
migration_data_dir = Path(os.getenv("MIGRATION_DATA_DIR", "."))
blob_path = migration_data_dir / "migration.blob"
yaml_path = migration_data_dir / "migration.yaml"

print(f"Loading migration data from {str(blob_path.resolve())}")

In [ ]:
res = client.load_migration_data(blob_path)
assert isinstance(res, sy.SyftSuccess), res.message

## Notes on workerpool upgrades
- We always skip default pool, since it is hardcoded as backend baseimage and cannot be changed by the user.
- Upgrade prebuilt pool: just enter your new tag.
- Upgrade syft-built pool: edit dockerfile string, rebuild.
- new pool is launched with same name, num_workers, upgraded image.
- mode="auto" should only be used in testing scenarios, as it assumes all custom image tags use syft versioning

In [ ]:
# NOTE do not use mode="auto" if you use custom image versioning.

sy.upgrade_custom_workerpools(client, blob_path, mode="auto")

# Post migration tests

In [ ]:
assert len(client.worker_pools.get_all()) == 3

In [ ]:
assert len(client.users.get_all()) == 2

In [ ]:
client_ds = server.login(email="ds@openmined.org", password="pw")

In [ ]:
req1 = client.requests[0]
req2 = client_ds.requests[0]
assert req1.status.name == "APPROVED" and req2.status.name == "APPROVED"
assert isinstance(req1._repr_html_(), str)
assert isinstance(req2._repr_html_(), str)

In [ ]:
jobs = client_ds.jobs
assert isinstance(jobs[0]._repr_html_(), str)

In [ ]:
ds = client_ds.datasets
asset = ds[0].assets[0]

In [ ]:
res = client_ds.code.compute_mean(data=asset, blocking=False)

In [ ]:
assert res.wait().get().shape == (100_000, 5)

In [ ]:
jobs = client_ds.jobs.get_all()

In [ ]:
job = jobs[0]

In [ ]:
job.logs()

In [ ]:
logs = job.logs(_print=False)
assert isinstance(logs, str)

In [ ]:
if server.server_type.value == "python":
    server.land()